# Background Images

Melissa Graham <br>
Last verified to run Wed Jul 20, with Weekly 2022_22.

This is a simple tutorial on how to retrieve, display, and characterize the subtracted backgrounds for `calexps` and `deepCoadds`.

#### Prerequisites

This tutorial assumes knowledge of afwDisplay and the Butler.
Tutorials that focus on image display and the Butler can be found in the <a href="https://github.com/rubin-dp0/tutorial-notebooks">rubin-dp0/tutorial-notebooks</a> repository.
(For image display, see NBs 03a, 03b, and 06a; for the Butler, see NB 04).

## Introduction to Background Subtraction

An overview of how the LSST Science Pipelines treats background subtraction can be found in the presentation by Yusra AlSayaad on Wed Mar 18, 2020 at the  <a href="https://project.lsst.org/meetings/law">LSST Algorithms Workshop</a> (scroll down on that page and find the link to the slides and a recording in the workshop agenda).

As described in Bosch et al. (2019), *An Overview of the LSST Image Processing Pipelines*, individual processed visit images (PVIs or `calexps`) are background-subtracted prior to coaddition, and then the final (small) background is subtracted from the coadded image. This final background can be *negative* to correct for oversubtraction. 

As of 2022, the best description of background subtraction in the image coaddition process can be found in Section 5.2.2 BackgroundMatchAndReject of the Data Management Science Pipelines
Design document at <a href="ls.st/ldm-151">ls.st/ldm-151</a>.

There is also an example of how to estimate the background with the LSST Science Pipelines <a href="https://pipelines.lsst.io/modules/lsst.afw.math/Background-example.html">here</a>, which might be of interest to some people.

In the Butler there are five `DatasetTypes` relevant to images and their backgrounds:

* Data Set (`DatasetType`) -- Description <br>
* `calexp` (`ExposureF`) -- Processed visit image with the background subtracted. <br>
* `calexpBackground` (`Background`) -- The background subtracted from `calexp`. <br>
* `deepCoadd` (`ExposureF`) -- The deep coaddition of the `calexps`. <br>
* `deepCoadd_calexp` (`ExposureF`) -- The deep coaddition of the `calexps`, with a final (small) background subtracted. <br>
* `deepCoadd_calexp_background` (`Background`) -- The background subtracted from `deepCoadd_calexp`.

Notice that the `calexps`, `deepCoadds`, and `deepCoadd_calexp` have a `DatasetType` = `ExposureF`, whereas the backgrounds `calexpBackground` and `deepCoadd_calexp_background` have `DatasetType` = `Background`.
When backgrounds are retrieved, the extra step `.getImage()` is necessary to obtain the subtracted background as an image. This will be demonstrated below.

<br>

## Set Up

#### Import packages.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings


from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

#### Set parameters.

In [ ]:
warnings.simplefilter("ignore", category=UserWarning)
afwDisplay.setDefaultBackend('matplotlib') 

#### Instantiate the Butler.

> **Note:** A pink-background warning about a version mismatch between the CFITSIO header and a linked library is a known issue that will be resolved in future releases.

In [ ]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

#### Optional: explore the butler for Background DatasetTypes.

Use the following to browse all `DatasetTypes`, included available image types and their `Backgrounds`. This is how a user would discover that `Background` is an available data type.

In [ ]:
# registry = butler.registry
# for x in sorted(registry.queryDatasetTypes()):
#     print(x)

<br>
<br>

## calexps

#### Specify a visit, retrieve calexp image from the Butler.

In [ ]:
calexpId = {'visit': '703697', 'detector': 80}#, 'band': 'g'}
calexp = butler.get('calexp', dataId=calexpId)

#### Plot the calexp.

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)
plt.gca().axis('off')

#### Retrieve and plot the background that was subtracted.

In [ ]:
bg_calexp = butler.get('calexpBackground', calexpId)

In [ ]:
bg_calexp_image = bg_calexp.getImage()

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.mtv(bg_calexp_image)

#### Get the array of pixel values and print statistics for the background.

Flatten the array and check for NaN values, too. Use numpy for basic array statistics.

In [ ]:
array = np.asarray( bg_calexp_image.getArray(), dtype='float' )

In [ ]:
temp = array.flatten()

In [ ]:
tx = np.where( np.isnan(temp) )[0]
print('There are ',len(tx),' NaN elements of temp.')

In [ ]:
print(len(temp))

print(np.min(temp), \
      np.max(temp), \
      np.mean(temp), \
      np.median(temp), \
      np.std(temp) )

#### Plot a histogram of the pixel values for the background.

In [ ]:
plt.hist( temp, bins=20, range=(np.nanmin(temp)-0.001,np.nanmax(temp)+0.001) )
plt.axvline( np.mean(temp), color='orange' )
plt.show()

#### Clean up.

In [ ]:
del temp, tx, array, bg_calexp_image, bg_calexp, calexp, calexpId

<br>
<br>

## deepCoadds

Recall that the backgrounds are subtracted prior to coaddition, and then a small final residual background is subtracted from the `deepCoadd_calexp`.
In other words, the background of the `deepCoadd_calexp` does not represent the combined backgrounds of the individual images that were combined.

#### Specify a tract and patch, and retrieve deepCoadd_calexp image from the Butler.

In [ ]:
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd_calexp', coaddId)

#### Optional: which visits went into constructing this coadd?

In [ ]:
# coaddInfo = coadd.getInfo()
# coaddVisits = coaddInfo.getCoaddInputs().visits
# coaddVisits.asAstropy()

#### Plot the deepCoadd_calexp image.

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(coadd.image)
plt.gca().axis('off')

#### Retrieve and plot the background that was subtracted from the deepCoadd_calexp image.

In [ ]:
bg_coadd = butler.get('deepCoadd_calexp_background', coaddId)

In [ ]:
bg_coadd_image = bg_coadd.getImage()

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
# afw_display.scale('linear', 'zscale')
afw_display.mtv(bg_coadd_image)

#### Get the array of pixel values and print statistics for the background of the deepCoadd_calexp image.

Flatten the array and check for NaN values, too. Use numpy for basic array statistics.

In [ ]:
array = np.asarray( bg_coadd_image.getArray(), dtype='float' )

In [ ]:
temp = array.flatten()

In [ ]:
tx = np.where( np.isnan(temp) )[0]
print('There are ',len(tx),' NaN elements of temp.')

In [ ]:
print(len(temp))

print(np.min(temp), \
      np.max(temp), \
      np.mean(temp), \
      np.median(temp), \
      np.std(temp) )

Since the statistics show the subtracted background is a uniform array of small values (i.e., the standard deviation in the pixel values is 0.0), making a histogram as done above for the `calexp` is not informative.

#### Show that deepCoadd_calexp - deepCoadd = deepCoadd_calexp_background

In [ ]:
coadd2Id = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd2 = butler.get('deepCoadd', coadd2Id)

In [ ]:
temp2 = coadd2.image.array.flatten() - coadd.image.array.flatten()

In [ ]:
print(len(temp2))

print(np.min(temp2), \
      np.max(temp2), \
      np.mean(temp2), \
      np.median(temp2), \
      np.std(temp2) )

These values are about the same as the background values above. 

#### Clean up

In [ ]:
del temp, tx, array, bg_coadd_image, bg_coadd, coadd, coaddId
del coadd2Id, coadd2, temp2

<br>

### To Do

Want to help improve this notebook or make a follow-up notebook? Consider exploring some of the following.

1. How similar/different are the calexp backgrounds? Retrieve all of the calexp backgrounds, calculate the standard deviation of background values by coordinate, and display that as an image.
2. Why isn't `deepCoadd_calexp` - `deepCoadd` *exactly the same as* `deepCoadd_calexp_background`? We show above the values are similar, but take a deeper dive and figure out why (e.g., edge effects?).